<a href="https://colab.research.google.com/github/harisreedhar/Swap-Mukham/blob/main/swap_mukham_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗿 **Swap-Mukham**
*Face swap app based on insightface inswapper.*
- [Github](https://github.com/harisreedhar/Swap-Mukham)
- [Disclaimer](https://github.com/harisreedhar/Swap-Mukham#disclaimer)

# Clone Repository

In [ ]:
#@title
! git clone https://github.com/harisreedhar/Swap-Mukham

# Install Requirements

In [ ]:
#@title
%cd Swap-Mukham/
print("Installing requirements...")
!pip install -r requirements.txt -q
!pip install gdown
print("Installing requirements done.")

# Download Models

In [ ]:
#@title
inswapper_model = "https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx" #@param {type:"string"}
gfpgan_model = "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth" #@param {type:"string"}
face_parser_model = "https://drive.google.com/uc?id=154JgKpzCPW82qINcVieuPH3fZ2e0P812" #@param {type:"string"}
real_esrgan_2x_model = "https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth" #@param {type:"string"}
real_esrgan_4x_model = "https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth" #@param {type:"string"}
real_esrgan_8x_model = "https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x8.pth" #@param {type:"string"}
codeformer_model = "https://huggingface.co/bluefoxcreation/Codeformer-ONNX/resolve/main/codeformer.onnx" #@param {type:"string"}
nsfw_det_model = "https://huggingface.co/bluefoxcreation/open-nsfw/resolve/main/open-nsfw.onnx" #@param {type:"string"}
import gdown
import urllib.request
print("Downloading swapper model...")
urllib.request.urlretrieve(inswapper_model, "/content/Swap-Mukham/assets/pretrained_models/inswapper_128.onnx")
print("Downloading gfpgan model...")
urllib.request.urlretrieve(gfpgan_model, "/content/Swap-Mukham/assets/pretrained_models/GFPGANv1.4.pth")
print("Downloading face parsing model...")
gdown.download(face_parser_model, "/content/Swap-Mukham/assets/pretrained_models/79999_iter.pth")
print("Downloading realesrgan 2x model...")
urllib.request.urlretrieve(real_esrgan_2x_model, "/content/Swap-Mukham/assets/pretrained_models/RealESRGAN_x2.pth")
print("Downloading realesrgan 4x model...")
urllib.request.urlretrieve(real_esrgan_4x_model, "/content/Swap-Mukham/assets/pretrained_models/RealESRGAN_x4.pth")
print("Downloading realesrgan 8x model...")
urllib.request.urlretrieve(real_esrgan_8x_model, "/content/Swap-Mukham/assets/pretrained_models/RealESRGAN_x8.pth")
print("Downloading codeformer...")
urllib.request.urlretrieve(codeformer_model, "/content/Swap-Mukham/assets/pretrained_models/codeformer.onnx")
print("Downloading NSFW detector model...")
urllib.request.urlretrieve(nsfw_det_model, "/content/Swap-Mukham/assets/pretrained_models/open-nsfw.onnx")
print("Downloading models done.")

# Mount Google drive (optional)

In [ ]:
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')

# Run App



In [ ]:
#@title
default_output_path = "/content/Swap-Mukham" #@param {type:"string"}

command = f"python app.py --cuda --colab --out_dir {default_output_path}"
!{command}